# Cancer detection using CNN

## Introduction

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation


### The Problem

Briefly describe the challenge problem and NLP.

The purpose of this project is to participate in a (now closed) Kaggle competition [Histopathologic Cancer Detection](https://www.kaggle.com/competitions/histopathologic-cancer-detection) by creating an algorithm to classify metastatic cancer cells in small images taken from larger digital pathology scans. The Kaggle site describes this data as 
" a slightly modified version of the [PatchCamelyon (PCam) benchmark dataset](https://github.com/basveeling/pcam) (the original PCam dataset contains duplicate images due to its probabilistic sampling, however, the version presented on Kaggle does not contain duplicates)."

### The Data

Describe the size, dimension, structure, etc., of the data. 

## EDA

Show a few visualizations like histograms. Describe any data cleaning procedures. Based on your EDA, what is your plan of analysis? 

## Architecture

Describe your model architecture and reasoning for why you believe that specific architecture would be suitable for this problem. Compare multiple architectures and tune hyperparameters. 

## Results and Analysis

### Kaggle competition ranking

Submission file .csv format:

id,label
0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0
95596b92e5066c5c52466c90b69ff089b39f2737,0
248e6738860e2ebcf6258cdc1f32f299e0c76914,0
etc.

## Conclusion

Discuss and interpret results as well as learnings and takeaways. What did and did not help improve the performance of your models? What improvements could you try in the future?

## References

1. 